# Generating counterfactual explanations with any ML model

The goal of this notebook is to show how to generate CFs for ML models using frameworks other than TensorFlow or PyTorch. This is a work in progress and here we show a method to generate diverse CFs by three methods: 
1. Independent random sampling of features
2. Genetic algorithm
3. Querying a KD tree

We use scikit-learn models for demonstration.  

# 1. Independent random sampling of features

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in dice_ml.utils.helpers module.

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

## Training a custom ML model

Below, we build an Artificial Neural Network using *MLPClassifier* in scikit-learn. We try to use the same set of parameters as used in this advanced [notebook](DiCE_with_private_data.ipynb), however, there are other framework-dependent parameters that can't be easily ported, so the accuracy/performance of the two models will be different.

In [5]:
train_ohe, test_ohe = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train_ohe = train_ohe.loc[:, train_ohe.columns != 'income']
y_train_ohe = train_ohe.loc[:, train_ohe.columns == 'income']
X_test_ohe = test_ohe.loc[:, test_ohe.columns != 'income']
y_test_ohe = test_ohe.loc[:, test_ohe.columns == 'income']

In [6]:
mlp_ohe = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=20, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp_ohe.fit(X_train_ohe, y_train_ohe.values.ravel())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.001, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=20, learning_rate='constant',
              learning_rate_init=0.01, max_iter=20, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=17, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.2, verbose=False, warm_start=False)

In [7]:
# provide the trained ML model to DiCE's model object
backend = 'sklearn'
m_ohe = dice_ml.Model(model=mlp_ohe, backend=backend)

## Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m_ohe, method='random')

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [11]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,47.0,Self-Employed,Some-college,Married,Professional,-,-,19.0,0.326
1,66.0,Government,Prof-school,Separated,Other/Unknown,Other,Male,81.0,0.29
2,33.0,Government,Doctorate,Separated,Sales,Other,Male,50.0,0.479
3,66.0,-,Assoc,Married,Professional,-,-,64.0,0.453


It can be observed that the random sampling method produces less sparse CFs in contrast to current DiCE's implementation. The sparsity issue with random sampling worsens with increasing *total_CFs* 

Further, different sets of counterfactuals can be generated with different random seeds.

In [12]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", random_seed=9) # default ranomd see is 17

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [13]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,76.0,Self-Employed,Some-college,Married,Professional,-,-,-,0.356
1,69.0,Government,Prof-school,Separated,Sales,Other,Male,57.0,0.467
2,63.0,Government,Doctorate,Separated,Sales,Other,Male,80.0,0.343
3,33.0,Self-Employed,Some-college,Married,Professional,-,-,68.0,0.306


### Selecting the features to vary

When few features are fixed, random sampling is unable to generate valid CFs while we get valid diverse CFs with current DiCE.

In [14]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       features_to_vary=['workclass','education','occupation','hours_per_week'])

Only 0 (required 4) Diverse Counterfactuals found for the given configuation, perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 00 min 00 sec


In [15]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



0 counterfactuals found!


### Choosing feature ranges

Since the features are sampled randomly, they can freely vary across their range. In the below example, we show how range of continuous features can be controlled using *permitted_range* parameter that can now be passed during CF generation.

In [16]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       permitted_range={'age':[22,50],'hours_per_week':[40,60]})

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [17]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,27.0,Government,Doctorate,Separated,Sales,Other,Male,56.0,0.493
1,40.0,Government,Doctorate,Separated,Sales,Other,Male,58.0,0.421
2,32.0,Self-Employed,Some-college,Married,Professional,-,-,47.0,0.309
3,39.0,-,Assoc,Married,Professional,-,-,43.0,0.376


# 2. Genetic Algorithm

Here, we show how to use DiCE can be used to generate CFs for any ML model by using the genetic algorithm to find the best counterfactuals close to the query point. The genetic algorithm converges quickly, and promotes diverse counterfactuals. 

## Training a custom ML model

Currently, the genetic algorithm method works with scikit-learn models. Support for Tensorflow 1&2 and Pytorch will be implemented soon.

We label-encode the data instead of one-hot encoding here. We plan to replace label-encoding with a more meaningful transformation for categorical values to numerical values

In [18]:
train_lbl, test_lbl = d.split_data(d.normalize_data(d.label_encoded_data, encoding='label'))
X_train_lbl = train_lbl.loc[:, train_lbl.columns != 'income']
y_train_lbl = train_lbl.loc[:, train_lbl.columns == 'income']
X_test_lbl = test_lbl.loc[:, test_lbl.columns != 'income']
y_test_lbl = test_lbl.loc[:, test_lbl.columns == 'income']

In [19]:
mlp_lbl = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=20, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp_lbl.fit(X_train_lbl, y_train_lbl.values.ravel())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.001, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=20, learning_rate='constant',
              learning_rate_init=0.01, max_iter=20, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=17, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.2, verbose=False, warm_start=False)

In [20]:
# provide the trained ML model to DiCE's model object
backend = 'sklearn'
m_lbl = dice_ml.Model(model=mlp_lbl, backend=backend)

## Generate diverse counterfactuals

In [21]:
# initiate DiceGenetic
exp_genetic = dice_ml.Dice(d, m_lbl, method='genetic')

In [22]:
# generate counterfactuals
dice_exp_genetic = exp_genetic.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [23]:
dice_exp_genetic.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Other/Unknown,Assoc,Married,Other/Unknown,White,Female,45.0,0.0



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,42.0,Government,-,Divorced,Blue-Collar,-,-,54.0,1
1,39.0,-,-,Divorced,-,-,Male,60.0,1
2,37.0,-,-,Divorced,-,Other,-,-,1
3,37.0,-,-,Divorced,-,-,-,44.0,1


Additional tools such as selecting features to vary and choosing feature ranges will be available shortly

# 3. Querying a KD Tree

Here, we show how to use DiCE can be used to generate CFs for any ML model by finding the closest points in the dataset that give the output as the desired class. We do this efficiently by building KD trees for each class, and querying the KD tree of the desired class to find the k closest counterfactuals from the dataset. The idea behind finding the closest points from the training data itself is to ensure that the counterfactuals displayed are feasible.

## Training a custom ML model

Currently, the KD Tree method works with scikit-learn models. Support for Tensorflow 1&2 and Pytorch will be implemented soon.

In [24]:
# provide the trained ML model to DiCE's model object
backend = 'sklearn'
m = dice_ml.Model(model=mlp_ohe, backend=backend)

## Generate diverse counterfactuals

In [25]:
# initiate DiceKD
exp_KD = dice_ml.Dice(d, m, method='kdtree')

In [26]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [27]:
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24.0,-,Bachelors,Married,White-Collar,-,-,-,1
1,26.0,-,Bachelors,Married,Professional,-,-,-,1
2,26.0,-,Bachelors,Married,White-Collar,-,-,-,1
3,27.0,-,Bachelors,Married,Sales,-,Male,-,1


### Selecting the features to vary

Here, you can choose to get counterfactuals only from the training data, or allow for some other counterfactuals as well 

In [28]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       features_to_vary=['age', 'workclass','education','occupation','hours_per_week'],
                                       training_points_only = True)

Only 0 (required 4) Diverse Counterfactuals found for the given configuation, perhaps change the query instance or the features to vary... ; total time taken: 00 min 00 sec


In [29]:
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0



0 counterfactuals found!


As we see, there are no proximal counterfactuals found using just the training instances. Now, we can allow for other counterfactuals as well. 

In [30]:
# generate counterfactuals
dice_exp_KD = exp_KD.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       features_to_vary=['age', 'workclass','education','occupation','hours_per_week'],
                                       training_points_only = False)

0 Counterfactuals found so far. Moving on to non-training points
Diverse Counterfactuals found! total time taken: 00 min 13 sec


In [31]:
dice_exp_KD.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,32.0,Self-Employed,Prof-school,-,Professional,-,-,49.0,1
1,39.0,Self-Employed,Prof-school,-,Professional,-,-,50.0,1
2,47.0,Self-Employed,Prof-school,-,Professional,-,-,52.0,1
3,48.0,-,Doctorate,-,White-Collar,-,-,53.0,1


Additional tools such as selecting feature ranges will be available shortly